In [7]:
import math
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
import pandas as pd


# B-S portfolio_pricer

In [14]:
def black_scholes_option_pricer(S, K, T, r, q, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        option_price = S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)
    else:
        raise ValueError("Type d'option non valide. Utilisez 'call' ou 'put'.")

    return option_price

def option_greeks(S, K, T, r, q, sigma, option_type):
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    N_prime_d1 = norm.pdf(d1)

    delta = 0
    gamma = 0
    vega = 0
    theta = 0

    if option_type == 'call':
        delta = norm.cdf(d1)
        gamma = N_prime_d1 / (S * sigma * np.sqrt(T))
        vega = S * np.sqrt(T) * N_prime_d1
        theta = -(S * sigma * N_prime_d1) / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        delta = norm.cdf(d1) - 1
        gamma = N_prime_d1 / (S * sigma * np.sqrt(T))
        vega = S * np.sqrt(T) * N_prime_d1
        theta = -(S * sigma * N_prime_d1) / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * norm.cdf(-d2)

    return delta, gamma, vega, theta

def portfolio_option_pricer(options):
    total_price = 0
    total_delta = 0
    total_gamma = 0
    total_vega = 0
    total_theta = 0

    for option in options:
        option_price = black_scholes_option_pricer(option['S'], option['K'], option['T'],
                                                   option['r'], option['q'], option['sigma'], option['type'])
        total_price += option_price * option['quantity']

        delta, gamma, vega, theta = option_greeks(option['S'], option['K'], option['T'],
                                                  option['r'], option['q'], option['sigma'], option['type'])

        total_delta += delta * option['quantity']
        total_gamma += gamma * option['quantity']
        total_vega += vega * option['quantity']
        total_theta += theta * option['quantity']

    return total_price, total_delta, total_gamma, total_vega, total_theta

# Exemple d'utilisation
options_portfolio = [
    {'S': 100, 'K': 110, 'T': 1, 'r': 0.05, 'q' : 0.07, 'sigma': 0.2, 'type': 'call', 'quantity': 2},
    {'S': 100, 'K': 90, 'T': 1, 'r': 0.05, 'q' : 0.07, 'sigma': 0.2, 'type': 'put', 'quantity': 1},
    # Ajoutez d'autres options au besoin
]

total_portfolio_price, total_portfolio_delta, total_portfolio_gamma, total_portfolio_vega, total_portfolio_theta = portfolio_option_pricer(options_portfolio)

print(f"Prix total du portefeuille d'options : {total_portfolio_price:.2f}")
print(f"Delta total du portefeuille : {total_portfolio_delta:.4f}")
print(f"Gamma total du portefeuille : {total_portfolio_gamma:.4f}")
print(f"Vega total du portefeuille : {total_portfolio_vega:.4f}")
print(f"Theta total du portefeuille : {total_portfolio_theta:.4f}")

Prix total du portefeuille d'options : 10.85
Delta total du portefeuille : 0.3345
Gamma total du portefeuille : 0.0530
Vega total du portefeuille : 105.9493
Theta total du portefeuille : -11.6120
